## Partie 1 : Word count dans un fichier txt.

Charger le fichier mobyDick.txt (précédemment uploadé) dans un RDD.

In [0]:
moby_rdd = sc.textFile("/FileStore/tables/mobyDick.txt")

Compter et afficher le nombre de lignes de celui-ci.

In [0]:
moby_rdd.count()

Out[3]: 21933

Créer un nouveau RDD qui contiendra les lignes contenant le mot ‘chapter’ (non sensible à la casse). Combien y en a-t-il ?

In [0]:
chapter_rdd = moby_rdd.filter(lambda x: "chapter" in x.lower().split())
print(chapter_rdd.count())


296


Créer un nouveau RDD qui contiendra uniquement les lignes non vides de mobyDick. Combien y en a-t-il ?


In [0]:
non_empty_rdd = moby_rdd.filter(lambda x: len(x.strip()) != 0)

Compter le nombre d’occurrences de chaque mot dans le document obtenu à la question précédente, en suivant l’approche du WordCount de MapReduce. Consulter la documentation des méthodes flatMap, map et reduceByKey. Combien y a-t-il de mots différents au total ?

In [0]:
word_counts = non_empty_rdd.flatMap(str.split).map(lambda word: (word, 1)).reduceByKey(lambda value1, value2: value1 + value2)
print(word_counts.count())

33086


Afficher les 10 mots les plus fréquents du livre.

In [0]:
word_counts.map(lambda tuple_mot_comptage: (tuple_mot_comptage[1], tuple_mot_comptage[0])).top(10)

Out[14]: [(13694, 'the'),
 (6531, 'of'),
 (5932, 'and'),
 (4493, 'a'),
 (4459, 'to'),
 (3850, 'in'),
 (2679, 'that'),
 (2428, 'his'),
 (1723, 'I'),
 (1649, 'with')]

## Partie 2 : Représentation d'un graphe sous forme de RDD (sans utiliser GraphX ni de GraphFrame).

On considère l’ensemble de triplets suivants, représentant les arcs d'un graphe :

In [0]:
graph = {
    (1, 0, 5),
    (5, 1, 8),
    (8, 2, 1),
    (2 ,0 ,6),
    (3, 0, 6),
    (6, 1, 9),
    (5, 1, 9),
    (9, 3, 11),
    (9, 4, 12),
    (4, 0, 7),
    (7, 1, 9),
    (7, 2, 10),
    (14, 1, 15),
    (15, 1, 16),
    (14, 1, 16),
    (17, 0, 18),
    (18, 0, 19),
    (19, 1, 20),
    (20, 0, 17),
}

La structure de chaque arc est la suivante :
    <br>- le premier élément correspond à l’identifiant du nœud d'origine (sujet du triplet),
    <br>- le second élément correspond au label de l’arc,
    <br>- le troisième et dernier élément correspond à l’identifiant du nœud d'arrivée (objet du triplet).

Charger le graphe dans un RDD. Compter et afficher le nombre d'arcs qu'il contient.

In [0]:
triplet_rdd = sc.parallelize(graph)
print(triplet_rdd.count())

19


En pratique nous ne souhaitons pas utiliser les labels des arcs. Créer un nouveau RDD ne comportant que les paires (sujet, objet).

In [0]:
so_pair_rdd = triplet_rdd.map(lambda x: (x[0], x[2]))

On appelle racine un noeud qui ne reçoit aucun arc (qui n'est objet d'aucun triplet). Créer un RDD ne contenant que les racines du graphe. Pour cela, créer deux rdd intermédiaires contenant l'ensemble des objets d'une part et des sujets d'autre part et les persister au niveau MEMORY_ONLY. Les racines sont les sujets privés des objets.
Afficher les racines.

In [0]:
from pyspark import StorageLevel

In [0]:
subjects = so_pair_rdd.map(lambda x: x[0]).distinct()
subjects.persist(StorageLevel.MEMORY_ONLY)
objects = so_pair_rdd.map(lambda x: x[1]).distinct()
objects.persist(StorageLevel.MEMORY_ONLY)

Out[8]: PythonRDD[30] at RDD at PythonRDD.scala:58

In [0]:
roots = subjects.subtract(objects)
roots.collect()

Out[11]: [2, 3, 4, 14]

De manière analogue, calculer et afficher les feuilles, noeuds qui ne sont origines (sujets) d'aucun arc.

In [0]:
leaves = objects.subtract(subjects)
leaves.collect()

Out[12]: [16, 10, 11, 12]

Question optionnelle, plus difficile :
Créer un nouveau RDD qui contient la "fermeture transitive" du graphe, correspondant à l'ensemble des paires (origine, destination) où la destination est accessible à partir de l'origine à partir d'un chemin composé d'un ou plusieurs arc.
Par exemple, si le graphe était {(1, 2), (2, 3)}, alors sa fermeture transitive serait {(1, 2), (2, 3), (1, 3)}, car 2 est accessible directement à partir de 1, 3 est accessible directement à partir de 2, et 3 est accessible indirectement à partir de 1, en deux étapes.


In [0]:
def transitive_closure(so_pair_rdd):
    
    def loop(old_pair_rdd, old_size):
        new_pair_rdd = old_pair_rdd.map(
            lambda x: (x[1], x[0])
        ).join(
            old_pair_rdd
        ).map(
            lambda x: x[1]
        ).union(
            old_pair_rdd
        ).distinct()
        new_size = new_pair_rdd.count()
        if old_size != new_size:
            return loop(new_pair_rdd, new_size)
        return new_pair_rdd
    
    return loop(so_pair_rdd, so_pair_rdd.count())

In [0]:
tc_rdd = transitive_closure(so_pair_rdd)
tc_rdd.collect()

Out[14]: [(20, 20),
 (7, 9),
 (1, 5),
 (1, 11),
 (4, 12),
 (14, 16),
 (5, 8),
 (2, 9),
 (7, 12),
 (5, 11),
 (8, 11),
 (20, 17),
 (17, 20),
 (8, 8),
 (4, 9),
 (9, 12),
 (5, 1),
 (8, 5),
 (7, 11),
 (1, 9),
 (17, 18),
 (5, 9),
 (19, 18),
 (18, 19),
 (1, 1),
 (19, 17),
 (5, 12),
 (18, 20),
 (15, 16),
 (20, 19),
 (3, 9),
 (1, 12),
 (4, 11),
 (6, 11),
 (5, 5),
 (19, 20),
 (8, 1),
 (17, 19),
 (2, 6),
 (3, 12),
 (18, 17),
 (9, 11),
 (19, 19),
 (2, 11),
 (6, 12),
 (3, 11),
 (8, 12),
 (18, 18),
 (4, 7),
 (2, 12),
 (14, 15),
 (20, 18),
 (8, 9),
 (6, 9),
 (3, 6),
 (4, 10),
 (7, 10),
 (1, 8),
 (17, 17)]

Déterminer et afficher les paires (sujet, objet) qui ont été ajoutées dans la fermeture transitive, en les triant par ordre croissant de sujet puis objet.

In [0]:
tc_rdd.subtract(so_pair_rdd).sortByKey(ascending=True).collect()

Out[19]: [(1, 9),
 (1, 8),
 (1, 1),
 (1, 11),
 (1, 12),
 (2, 11),
 (2, 12),
 (2, 9),
 (3, 9),
 (3, 12),
 (3, 11),
 (4, 12),
 (4, 11),
 (4, 9),
 (4, 10),
 (5, 12),
 (5, 5),
 (5, 11),
 (5, 1),
 (6, 11),
 (6, 12),
 (7, 11),
 (7, 12),
 (8, 12),
 (8, 9),
 (8, 11),
 (8, 8),
 (8, 5),
 (17, 17),
 (17, 19),
 (17, 20),
 (18, 20),
 (18, 17),
 (18, 18),
 (19, 19),
 (19, 18),
 (19, 17),
 (20, 19),
 (20, 18),
 (20, 20)]

Créer un RDD contenant l'ensemble des nœuds accessibles à partir d'une racine.
Chaque élément de ce RDD contiendra un tuple avec la racine en première position et une liste triée de tous les nœuds accessibles en deuxième position.

In [0]:
# Votre code ici